# Import Libraries and Data

In [ ]:
import pandas as pd
import numpy as np
import math
import glob
import os
import sys
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import KNNImputer
from sklearn.neighbors import LocalOutlierFactor
# import ipynb.fs.full.functions as func # importing functions from functions module

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#pd.options.display.max_rows = 999
pd.set_option("display.max_columns", None)

In [ ]:
# for working from google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install import-ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 2.1 MB/s 


In [ ]:
import import_ipynb
import sys
sys.path.append('/content/drive/MyDrive/Master/Tez')
%cd "/content/drive/MyDrive/Master/Tez"

/content/drive/MyDrive/Master/Tez


In [ ]:
import functions as func

importing Jupyter notebook from functions.ipynb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# for jupyter notebook
# path = r'C:\Users\Lenovo\Desktop\ML Neuro\data\Data for ML_v2.xlsx'

In [ ]:
# df = pd.read_excel(path, engine='openpyxl')
df = pd.read_excel('Data for ML_v2.xlsx', engine='openpyxl')

# Preliminary Cleaning

In [ ]:
func.missing_values_table(df)

Your selected dataframe has 137 columns.
There are 133 columns that have missing values.


,Missing Values,% of Total Values
Khbt16,511,30.4
Khbo16,511,30.4
Koxy16,511,30.4
Khbr16,511,30.4
Ghbr16,509,30.3
...,...,...
Ghbt5,2,0.1
Khbo7,1,0.1
Khbt7,1,0.1
Koxy7,1,0.1


## Count Nan values in Response feature then creating new df by removing them.

In [ ]:
count_Nan = df.Response.isna().sum()
count_Nan
df_last = df[df['Response'].notna()]

114

## Checking if all removed values are Nan

In [ ]:
df_last.shape
df.shape[0] - df_last.shape[0] == count_Nan

(1566, 137)

True

## Removing values other than left and right in "Response" feature

In [ ]:
response = ['Left', 'Right']
df_last = df_last[df_last.Response.isin(response)]
df_last.reset_index(inplace = True, drop = True) # resetting indexing
df_last.shape

(1565, 137)

In [ ]:
func.missing_values_table(df_last)

Your selected dataframe has 137 columns.
There are 128 columns that have missing values.


,Missing Values,% of Total Values
Khbt16,503,32.1
Khbr16,503,32.1
Koxy16,503,32.1
Khbo16,503,32.1
Ghbo16,501,32.0
...,...,...
Ghbr5,2,0.1
Koxy7,1,0.1
Khbo7,1,0.1
Khbt7,1,0.1


## Checking if there are Nan values left in non measurement features and then count missing measurement values

In [ ]:
df_last.isna().sum()
df_last.isna().sum().sum()

Part_No        0
Stim_ID        0
Y/K            0
Direction      0
RespTime       0
            ... 
Khbt12        31
Khbt13        29
Khbt14       106
Khbt15       454
Khbt16       503
Length: 137, dtype: int64

13718

## Removing rows that have missing value percentage more than 25% for each G and K measurements

In [ ]:
df_last_GmissingValues = func.missingValuesRows(df_last.iloc[:,9:73])

Your selected dataframe has 1565 columns.
There are 934 columns that have missing values.


In [ ]:
df_last_GmissingValues[df_last_GmissingValues['Missing Values'] >= 15].min()

Missing Values       16.0
% of Total Values    25.0
dtype: float64

In [ ]:
df_last_GmissingValues[df_last_GmissingValues['Missing Values'] >= 15].count()

Missing Values       47
% of Total Values    47
dtype: int64

In [ ]:
df_last_KmissingValues = func.missingValuesRows(df_last.iloc[:,73:])

Your selected dataframe has 1565 columns.
There are 933 columns that have missing values.


In [ ]:
df_last_KmissingValues[df_last_KmissingValues['Missing Values'] >= 15].min()

Missing Values       16.0
% of Total Values    25.0
dtype: float64

In [ ]:
df_last_KmissingValues[df_last_KmissingValues['Missing Values'] >= 15].count()

Missing Values       45
% of Total Values    45
dtype: int64

In [ ]:
subset_1 = df_last.columns[9:73]
subset_2 = df_last.columns[73:]
df_last = df_last.dropna(thresh=49, subset=subset_1)
df_last = df_last.dropna(thresh=49, subset=subset_2)

In [ ]:
df_last.reset_index(drop=True)

,Part_No,Stim_ID,Y/K,Direction,RespTime,Response,Like_OR_Nolike,G1,G2,Goxy1,Goxy2,Goxy3,Goxy4,Goxy5,Goxy6,Goxy7,Goxy8,Goxy9,Goxy10,Goxy11,Goxy12,Goxy13,Goxy14,Goxy15,Goxy16,Ghbr1,Ghbr2,Ghbr3,Ghbr4,Ghbr5,Ghbr6,Ghbr7,Ghbr8,Ghbr9,Ghbr10,Ghbr11,Ghbr12,Ghbr13,Ghbr14,Ghbr15,Ghbr16,Ghbo1,Ghbo2,Ghbo3,Ghbo4,Ghbo5,Ghbo6,Ghbo7,Ghbo8,Ghbo9,Ghbo10,Ghbo11,Ghbo12,Ghbo13,Ghbo14,Ghbo15,Ghbo16,Ghbt1,Ghbt2,Ghbt3,Ghbt4,Ghbt5,Ghbt6,Ghbt7,Ghbt8,Ghbt9,Ghbt10,Ghbt11,Ghbt12,Ghbt13,Ghbt14,Ghbt15,Ghbt16,Koxy1,Koxy2,Koxy3,Koxy4,Koxy5,Koxy6,Koxy7,Koxy8,Koxy9,Koxy10,Koxy11,Koxy12,Koxy13,Koxy14,Koxy15,Koxy16,Khbr1,Khbr2,Khbr3,Khbr4,Khbr5,Khbr6,Khbr7,Khbr8,Khbr9,Khbr10,Khbr11,Khbr12,Khbr13,Khbr14,Khbr15,Khbr16,Khbo1,Khbo2,Khbo3,Khbo4,Khbo5,Khbo6,Khbo7,Khbo8,Khbo9,Khbo10,Khbo11,Khbo12,Khbo13,Khbo14,Khbo15,Khbo16,Khbt1,Khbt2,Khbt3,Khbt4,Khbt5,Khbt6,Khbt7,Khbt8,Khbt9,Khbt10,Khbt11,Khbt12,Khbt13,Khbt14,Khbt15,Khbt16
0,1,36.0,0.0,2.0,2292.0,Left,True,1,1,0.530860,0.571811,0.876945,0.895395,0.618246,0.775132,1.141467,0.017443,0.231609,-0.084973,0.011109,-0.082605,-1.784506,-0.598040,-0.671195,NaN,-0.439243,-0.498822,-0.564051,-0.807433,-0.413861,-0.650190,-0.978439,-0.074692,-0.243569,0.039038,0.121672,0.054662,1.092793,0.388220,0.384527,NaN,0.091617,0.072988,0.312894,0.087962,0.204385,0.124942,0.163028,-0.057249,-0.011960,-0.045935,0.132781,-0.027944,-0.691713,-0.209819,-0.286668,NaN,-0.347626,-0.425834,-0.251157,-0.719471,-0.209475,-0.525248,-0.815411,-0.131941,-0.255529,-0.006897,0.254454,0.026718,0.401080,0.178401,0.097859,NaN,0.811057,1.100687,1.334711,1.311974,0.809818,0.952328,1.423814,0.018645,0.260361,-0.240256,-0.078202,-0.035370,-2.348474,-0.714057,-0.741101,NaN,-0.646730,-0.564837,-0.882807,-0.951400,-0.544274,-0.603159,-1.220996,0.072544,-0.353696,0.268320,0.150834,0.116103,1.461363,0.591867,0.453544,NaN,0.164326,0.535850,0.451904,0.360574,0.265544,0.349169,0.202817,0.091189,-0.093334,0.028064,0.072632,0.080733,-0.887111,-0.122190,-0.287557,NaN,-0.482404,-0.028987,-0.430903,-0.590826,-0.278730,-0.253990,-1.018179,0.163734,-0.447030,0.296384,0.223466,0.196836,0.574252,0.469677,0.165987,NaN
1,1,16.0,0.0,2.0,1459.0,Right,False,1,2,-0.016150,0.089965,-0.148061,0.206889,0.070859,0.378170,0.412107,0.325312,0.439381,0.417096,0.303118,0.397769,0.629696,0.427812,0.483460,0.208160,-0.048277,-0.208151,0.119465,-0.232437,-0.011533,-0.272199,-0.206342,-0.476173,-0.276975,-0.575903,-0.143816,-0.315825,-0.523744,-0.599519,-0.428484,-0.316551,-0.064427,-0.118186,-0.028596,-0.025547,0.059326,0.105971,0.205765,-0.150861,0.162406,-0.158807,0.159301,0.081944,0.105952,-0.171707,0.054975,-0.108391,-0.112703,-0.326338,0.090868,-0.257984,0.047793,-0.166229,-0.000578,-0.627035,-0.114569,-0.734710,0.015485,-0.233881,-0.417792,-0.771225,-0.373509,-0.424942,0.174428,0.653096,-0.038715,0.650251,0.175557,0.766687,0.819796,0.545715,0.706113,0.635638,0.500461,0.649931,0.412645,0.467408,0.471701,1.463744,-0.249283,-0.731326,0.160259,-0.528862,-0.040307,-0.479963,-0.354216,-0.706690,-0.334903,-0.812840,-0.184560,-0.441936,-0.368731,-0.705205,-0.498745,-2.332100,-0.074855,-0.078230,0.121545,0.121389,0.135250,0.286725,0.465580,-0.160975,0.371210,-0.177202,0.315901,0.207995,0.043914,-0.237797,-0.027044,-0.868356,-0.324137,-0.809556,0.281804,-0.407472,0.094943,-0.193238,0.111364,-0.867665,0.036307,-0.990043,0.131340,-0.233941,-0.324817,-0.943003,-0.525789,-3.200456
2,1,42.0,0.0,2.0,970.0,Left,True,1,3,-0.136447,-0.272634,-0.322293,-0.284869,-0.168651,-0.220061,-0.561465,-0.030628,-0.085240,0.279583,0.185449,0.248090,0.449046,0.604036,0.395534,0.301781,0.115040,0.660709,0.119293,0.232511,0.163833,0.103896,0.329422,0.097941,0.002345,-0.065501,-0.122785,-0.246533,-0.138044,-0.383889,-0.125328,-0.081074,-0.021407,0.388075,-0.203000,-0.052358,-0.004818,-0.116165,-0.232043,0.067314,-0.082895,0.214081,0.062664,0.001557,0.311002,0.220147,0.270206,0.220707,0.093633,1.048784,-0.083707,0.180153,0.159015,-0.012269,0.097379,0.165255,-0.080550,0.148580,-0.060121,-0.244976,0.172957,-0.163742,0.144877,0.

In [ ]:
df_last_notLike = df_last[df_last['Like_OR_Nolike'] == False]

In [ ]:
df_last_notLike

,Part_No,Stim_ID,Y/K,Direction,RespTime,Response,Like_OR_Nolike,G1,G2,Goxy1,Goxy2,Goxy3,Goxy4,Goxy5,Goxy6,Goxy7,Goxy8,Goxy9,Goxy10,Goxy11,Goxy12,Goxy13,Goxy14,Goxy15,Goxy16,Ghbr1,Ghbr2,Ghbr3,Ghbr4,Ghbr5,Ghbr6,Ghbr7,Ghbr8,Ghbr9,Ghbr10,Ghbr11,Ghbr12,Ghbr13,Ghbr14,Ghbr15,Ghbr16,Ghbo1,Ghbo2,Ghbo3,Ghbo4,Ghbo5,Ghbo6,Ghbo7,Ghbo8,Ghbo9,Ghbo10,Ghbo11,Ghbo12,Ghbo13,Ghbo14,Ghbo15,Ghbo16,Ghbt1,Ghbt2,Ghbt3,Ghbt4,Ghbt5,Ghbt6,Ghbt7,Ghbt8,Ghbt9,Ghbt10,Ghbt11,Ghbt12,Ghbt13,Ghbt14,Ghbt15,Ghbt16,Koxy1,Koxy2,Koxy3,Koxy4,Koxy5,Koxy6,Koxy7,Koxy8,Koxy9,Koxy10,Koxy11,Koxy12,Koxy13,Koxy14,Koxy15,Koxy16,Khbr1,Khbr2,Khbr3,Khbr4,Khbr5,Khbr6,Khbr7,Khbr8,Khbr9,Khbr10,Khbr11,Khbr12,Khbr13,Khbr14,Khbr15,Khbr16,Khbo1,Khbo2,Khbo3,Khbo4,Khbo5,Khbo6,Khbo7,Khbo8,Khbo9,Khbo10,Khbo11,Khbo12,Khbo13,Khbo14,Khbo15,Khbo16,Khbt1,Khbt2,Khbt3,Khbt4,Khbt5,Khbt6,Khbt7,Khbt8,Khbt9,Khbt10,Khbt11,Khbt12,Khbt13,Khbt14,Khbt15,Khbt16
1,1,16.0,0.0,2.0,1459.0,Right,False,1,2,-0.016150,0.089965,-0.148061,0.206889,0.070859,0.378170,0.412107,0.325312,0.439381,0.417096,0.303118,0.397769,0.629696,0.427812,0.483460,0.208160,-0.048277,-0.208151,0.119465,-0.232437,-0.011533,-0.272199,-0.206342,-0.476173,-0.276975,-0.575903,-0.143816,-0.315825,-0.523744,-0.599519,-0.428484,-0.316551,-0.064427,-0.118186,-0.028596,-0.025547,0.059326,0.105971,0.205765,-0.150861,0.162406,-0.158807,0.159301,0.081944,0.105952,-0.171707,0.054975,-0.108391,-0.112703,-0.326338,0.090868,-0.257984,0.047793,-0.166229,-0.000578,-0.627035,-0.114569,-0.734710,0.015485,-0.233881,-0.417792,-0.771225,-0.373509,-0.424942,0.174428,0.653096,-0.038715,0.650251,0.175557,0.766687,0.819796,0.545715,0.706113,0.635638,0.500461,0.649931,0.412645,0.467408,0.471701,1.463744,-0.249283,-0.731326,0.160259,-0.528862,-0.040307,-0.479963,-0.354216,-0.706690,-0.334903,-0.812840,-0.184560,-0.441936,-0.368731,-0.705205,-0.498745,-2.332100,-0.074855,-0.078230,0.121545,0.121389,0.135250,0.286725,0.465580,-0.160975,0.371210,-0.177202,0.315901,0.207995,0.043914,-0.237797,-0.027044,-0.868356,-0.324137,-0.809556,0.281804,-0.407472,0.094943,-0.193238,0.111364,-0.867665,0.036307,-0.990043,0.131340,-0.233941,-0.324817,-0.943003,-0.525789,-3.200456
3,1,1.0,1.0,1.0,1073.0,Left,False,1,4,-0.171196,-0.102954,-0.875261,-0.000342,-0.337183,0.217429,0.020822,0.454714,0.233263,0.839838,-0.104713,0.503555,0.035672,0.926225,0.184096,0.786021,-0.099402,-0.441360,0.420930,-0.323090,0.189353,-0.532296,-0.005857,-0.762548,-0.052403,-0.918665,0.075824,-0.709362,-0.087954,-1.035489,-0.209049,-1.158090,-0.270597,-0.544314,-0.454330,-0.323432,-0.147830,-0.314867,0.014965,-0.307834,0.180860,-0.078826,-0.028890,-0.205807,-0.052281,-0.109264,-0.024954,-0.372069,-0.369999,-0.985674,-0.033400,-0.646521,0.041522,-0.847163,0.009108,-1.070382,0.128457,-0.997491,0.046934,-0.915169,-0.140235,-1.144753,-0.234003,-1.530159,-0.213702,0.101851,-0.864369,-0.020777,-0.392224,0.134953,-0.208951,0.347925,0.067846,0.784468,-0.266378,0.443826,-0.229802,0.850748,0.155352,1.158873,-0.053198,-0.704420,0.395649,-0.437197,0.156076,-0.628296,-0.004996,-0.789858,-0.012509,-0.957821,0.107769,-0.761388,0.004438,-1.095664,-0.260718,-1.962783,-0.266899,-0.602569,-0.468720,-0.457974,-0.236148,-0.493343,-0.213947,-0.441933,0.055337,-0.173353,-0.158609,-0.317562,-0.225364,-0.244915,-0.105366,-0.803910,-0.320097,-1.306990,-0.073071,-0.895170,-0.080073,-1.121639,-0.218943,-1.231791,0.042828,-1.131174,-0.050840,-1.078950,-0.220926,-1.340579,-0.366084,-2.766693
5,1,13.0,1.0,2.0,1032.0,Right,False,1,6,-0.157769,-0.131881,-0.130702,-0.119899,0.183395,0.133742,0.663423,0.371626,0.649885,0.407279,0.481765,0.141346,0.053663,-0.086834,-0.090411,-0.435021,0.154087,-0.116972,0.097887,0.127719,0.004732,-0.079551,-0.238360,-0.067118,-0.121218,-0.010910,-0.137637,-0.069001,-0.002125,0.114249,0.125962,0.807547,-0.003681,-0.248854,-0.032814,0.007820,0.188127,0.054190,0.425062,0.304508,0.528668,0.396369,0.344127,0.072345,0.051538,0.027414,0.035551,0.372527,0.150406,-0.365826,0.065073,0.135538,0.192858,-0.025361,0.186702,0.

In [ ]:
df_last_Like = df_last[df_last['Like_OR_Nolike'] == True]

In [ ]:
df_last_Like

,Part_No,Stim_ID,Y/K,Direction,RespTime,Response,Like_OR_Nolike,G1,G2,Goxy1,Goxy2,Goxy3,Goxy4,Goxy5,Goxy6,Goxy7,Goxy8,Goxy9,Goxy10,Goxy11,Goxy12,Goxy13,Goxy14,Goxy15,Goxy16,Ghbr1,Ghbr2,Ghbr3,Ghbr4,Ghbr5,Ghbr6,Ghbr7,Ghbr8,Ghbr9,Ghbr10,Ghbr11,Ghbr12,Ghbr13,Ghbr14,Ghbr15,Ghbr16,Ghbo1,Ghbo2,Ghbo3,Ghbo4,Ghbo5,Ghbo6,Ghbo7,Ghbo8,Ghbo9,Ghbo10,Ghbo11,Ghbo12,Ghbo13,Ghbo14,Ghbo15,Ghbo16,Ghbt1,Ghbt2,Ghbt3,Ghbt4,Ghbt5,Ghbt6,Ghbt7,Ghbt8,Ghbt9,Ghbt10,Ghbt11,Ghbt12,Ghbt13,Ghbt14,Ghbt15,Ghbt16,Koxy1,Koxy2,Koxy3,Koxy4,Koxy5,Koxy6,Koxy7,Koxy8,Koxy9,Koxy10,Koxy11,Koxy12,Koxy13,Koxy14,Koxy15,Koxy16,Khbr1,Khbr2,Khbr3,Khbr4,Khbr5,Khbr6,Khbr7,Khbr8,Khbr9,Khbr10,Khbr11,Khbr12,Khbr13,Khbr14,Khbr15,Khbr16,Khbo1,Khbo2,Khbo3,Khbo4,Khbo5,Khbo6,Khbo7,Khbo8,Khbo9,Khbo10,Khbo11,Khbo12,Khbo13,Khbo14,Khbo15,Khbo16,Khbt1,Khbt2,Khbt3,Khbt4,Khbt5,Khbt6,Khbt7,Khbt8,Khbt9,Khbt10,Khbt11,Khbt12,Khbt13,Khbt14,Khbt15,Khbt16
0,1,36.0,0.0,2.0,2292.0,Left,True,1,1,0.530860,0.571811,0.876945,0.895395,0.618246,0.775132,1.141467,0.017443,0.231609,-0.084973,0.011109,-0.082605,-1.784506,-0.598040,-0.671195,NaN,-0.439243,-0.498822,-0.564051,-0.807433,-0.413861,-0.650190,-0.978439,-0.074692,-0.243569,0.039038,0.121672,0.054662,1.092793,0.388220,0.384527,NaN,0.091617,0.072988,0.312894,0.087962,0.204385,0.124942,0.163028,-0.057249,-0.011960,-0.045935,0.132781,-0.027944,-0.691713,-0.209819,-0.286668,NaN,-0.347626,-0.425834,-0.251157,-0.719471,-0.209475,-0.525248,-0.815411,-0.131941,-0.255529,-0.006897,0.254454,0.026718,0.401080,0.178401,0.097859,NaN,0.811057,1.100687,1.334711,1.311974,0.809818,0.952328,1.423814,0.018645,0.260361,-0.240256,-0.078202,-0.035370,-2.348474,-0.714057,-0.741101,NaN,-0.646730,-0.564837,-0.882807,-0.951400,-0.544274,-0.603159,-1.220996,0.072544,-0.353696,0.268320,0.150834,0.116103,1.461363,0.591867,0.453544,NaN,0.164326,0.535850,0.451904,0.360574,0.265544,0.349169,0.202817,0.091189,-0.093334,0.028064,0.072632,0.080733,-0.887111,-0.122190,-0.287557,NaN,-0.482404,-0.028987,-0.430903,-0.590826,-0.278730,-0.253990,-1.018179,0.163734,-0.447030,0.296384,0.223466,0.196836,0.574252,0.469677,0.165987,NaN
2,1,42.0,0.0,2.0,970.0,Left,True,1,3,-0.136447,-0.272634,-0.322293,-0.284869,-0.168651,-0.220061,-0.561465,-0.030628,-0.085240,0.279583,0.185449,0.248090,0.449046,0.604036,0.395534,0.301781,0.115040,0.660709,0.119293,0.232511,0.163833,0.103896,0.329422,0.097941,0.002345,-0.065501,-0.122785,-0.246533,-0.138044,-0.383889,-0.125328,-0.081074,-0.021407,0.388075,-0.203000,-0.052358,-0.004818,-0.116165,-0.232043,0.067314,-0.082895,0.214081,0.062664,0.001557,0.311002,0.220147,0.270206,0.220707,0.093633,1.048784,-0.083707,0.180153,0.159015,-0.012269,0.097379,0.165255,-0.080550,0.148580,-0.060121,-0.244976,0.172957,-0.163742,0.144877,0.139633,0.282411,0.064885,0.454139,0.259362,0.431352,0.287118,0.094820,0.439876,-0.284129,0.363272,-0.219946,0.308470,-0.543672,0.077329,0.059039,0.083751,-0.152114,0.542451,-0.182078,0.009384,0.039956,-0.004640,0.229157,-0.068781,0.531488,0.114320,0.479081,-0.079169,0.637054,-0.129024,0.096418,-0.135218,0.130297,0.607335,0.272061,0.268746,0.471308,0.282478,0.323977,0.371096,0.247360,0.477592,0.259134,0.229301,0.093382,-0.051695,0.155457,-0.051467,-0.021817,1.149786,0.089983,0.278130,0.511265,0.277837,0.553134,0.302315,0.778848,0.591912,0.738215,0.150131,0.730436,-0.180719,0.251875,-0.186686
4,1,46.0,0.0,2.0,745.0,Left,True,1,5,-0.212047,-0.366153,-0.342529,-0.316131,-0.423417,-0.224753,-0.476353,-0.021907,-0.269171,-0.060163,-0.368411,-0.162705,-0.365288,-0.052181,-0.324225,-0.934540,0.137627,0.467511,-0.023956,0.048424,0.147062,-0.023436,0.139948,-0.008951,-0.037714,-0.004652,0.027158,-0.129309,0.054662,-0.096996,0.184951,1.800187,-0.074420,0.101358,-0.366485,-0.267707,-0.276355,-0.248189,-0.336405,-0.030858,-0.306885,-0.064815,-0.341253,-0.292014,-0.310626,-0.149177,-0.139274,0.865647,0.063207,0.568869,-0.390440,-0.219283,-0.129293,-0.271625,-0.196457,-0.039808,-0.344599,-0.069466,-0.314095,-0.421323,-0.255964,-0.246174,0.045677,2.665835,-0.2

# Outlier Detection

## Outlier Removal with IQR

In [ ]:
df_last_notLike.describe()
df_last_Like.describe()

,Part_No,Stim_ID,Y/K,Direction,RespTime,G1,G2,Goxy1,Goxy2,Goxy3,Goxy4,Goxy5,Goxy6,Goxy7,Goxy8,Goxy9,Goxy10,Goxy11,Goxy12,Goxy13,Goxy14,Goxy15,Goxy16,Ghbr1,Ghbr2,Ghbr3,Ghbr4,Ghbr5,Ghbr6,Ghbr7,Ghbr8,Ghbr9,Ghbr10,Ghbr11,Ghbr12,Ghbr13,Ghbr14,Ghbr15,Ghbr16,Ghbo1,Ghbo2,Ghbo3,Ghbo4,Ghbo5,Ghbo6,Ghbo7,Ghbo8,Ghbo9,Ghbo10,Ghbo11,Ghbo12,Ghbo13,Ghbo14,Ghbo15,Ghbo16,Ghbt1,Ghbt2,Ghbt3,Ghbt4,Ghbt5,Ghbt6,Ghbt7,Ghbt8,Ghbt9,Ghbt10,Ghbt11,Ghbt12,Ghbt13,Ghbt14,Ghbt15,Ghbt16,Koxy1,Koxy2,Koxy3,Koxy4,Koxy5,Koxy6,Koxy7,Koxy8,Koxy9,Koxy10,Koxy11,Koxy12,Koxy13,Koxy14,Koxy15,Koxy16,Khbr1,Khbr2,Khbr3,Khbr4,Khbr5,Khbr6,Khbr7,Khbr8,Khbr9,Khbr10,Khbr11,Khbr12,Khbr13,Khbr14,Khbr15,Khbr16,Khbo1,Khbo2,Khbo3,Khbo4,Khbo5,Khbo6,Khbo7,Khbo8,Khbo9,Khbo10,Khbo11,Khbo12,Khbo13,Khbo14,Khbo15,Khbo16,Khbt1,Khbt2,Khbt3,Khbt4,Khbt5,Khbt6,Khbt7,Khbt8,Khbt9,Khbt10,Khbt11,Khbt12,Khbt13,Khbt14,Khbt15,Khbt16
count,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,671.000000,671.000000,663.000000,679.000000,715.000000,714.000000,715.000000,692.000000,715.000000,698.000000,715.000000,714.000000,710.000000,688.000000,540.000000,500.000000,668.000000,671.000000,663.000000,679.000000,715.000000,714.000000,715.000000,692.000000,715.000000,698.000000,715.000000,714.000000,710.000000,688.000000,540.000000,500.000000,668.000000,671.000000,663.000000,679.000000,715.000000,714.000000,715.000000,692.000000,715.000000,698.000000,715.000000,714.000000,710.000000,688.000000,540.000000,500.000000,668.000000,671.000000,663.000000,679.000000,715.000000,714.000000,715.000000,692.000000,715.000000,698.000000,715.000000,714.000000,710.000000,688.000000,540.000000,500.000000,667.000000,667.000000,663.000000,679.000000,715.000000,715.000000,715.000000,692.000000,715.000000,697.000000,715.000000,715.000000,710.000000,686.000000,540.000000,497.000000,667.000000,667.000000,663.000000,679.000000,715.000000,715.000000,715.000000,692.000000,715.000000,697.000000,715.000000,715.000000,710.000000,686.000000,540.000000,497.000000,667.000000,667.000000,663.000000,679.000000,715.000000,715.000000,715.000000,692.000000,715.000000,697.000000,715.000000,715.000000,710.000000,686.000000,540.000000,497.000000,667.000000,667.000000,663.000000,679.000000,715.000000,715.000000,715.000000,692.000000,715.000000,697.000000,715.000000,715.000000,710.000000,686.000000,540.000000,497.000000
mean,14.300699,29.590210,0.549650,1.464336,1114.732867,14.300699,31.474126,-0.024515,-0.017282,-0.043572,-0.013837,0.004042,-0.004778,0.006219,0.008333,0.007347,0.001727,0.000585,-0.025949,-0.041301,-0.033200,-0.021313,-0.017619,0.004252,0.014673,0.019735,0.009296,0.006730,0.009220,0.004143,-0.004144,0.008539,0.008241,0.010563,0.030802,0.023317,0.032452,0.008751,0.039195,-0.019609,-0.002609,-0.023837,-0.004541,0.010772,0.004443,0.010362,0.004189,0.015886,0.009968,0.011148,0.004852,-0.017984,-0.000748,-0.012561,0.021575,-0.015357,0.012064,-0.004102,0.004755,0.017502,0.013663,0.014505,0.000044,0.024425,0.018209,0.021710,0.035654,0.005333,0.031704,-0.003810,0.060770,0.025851,0.024108,-0.010776,0.009859,0.001739,0.005700,0.012459,0.000033,0.004454,-0.002791,0.003270,-0.012090,-0.028335,-0.023585,0.002994,0.009457,-0.008376,-0.010003,0.019264,0.004444,0.026534,0.018078,0.014647,0.018478,0.024895,0.025624,0.023779,0.023967,0.026895,0.024689,0.008335,0.015590,0.017474,0.014105,0.008488,0.014303,0.028273,0.023778,0.027106,0.018511,0.029349,0.022833,0.027049,0.011877,-0.001439,0.001104,0.011329,0.025048,0.009098,0.004102,0.027752,0.018746,0.054807,0.041856,0.041752,0.036988,0.054244,0.048457,0.050828,0.035844,0.025456,0.025793,0.019664,0.040638
std,8.140946,17.381569,0.497877,0.499076,425.148911,8.140946,17.207836,0.351909,0.280703,0.337261,0.263740,0.279396,0.257835,0.337287,0.259890,0.337122,0.251939,0.411799,0.257150,0.530331,0.289752,0.327355,0.264891,0.264462,0.259214,0.139998,0.199889,0.138595,0.209477,0.203041,0.286031,0.169700,0.286578,0.205469,0.256468,0.256007,0.277592,0.213251,0.383597,0.270688,0.270971,0.2613

,Part_No,Stim_ID,Y/K,Direction,RespTime,G1,G2,Goxy1,Goxy2,Goxy3,Goxy4,Goxy5,Goxy6,Goxy7,Goxy8,Goxy9,Goxy10,Goxy11,Goxy12,Goxy13,Goxy14,Goxy15,Goxy16,Ghbr1,Ghbr2,Ghbr3,Ghbr4,Ghbr5,Ghbr6,Ghbr7,Ghbr8,Ghbr9,Ghbr10,Ghbr11,Ghbr12,Ghbr13,Ghbr14,Ghbr15,Ghbr16,Ghbo1,Ghbo2,Ghbo3,Ghbo4,Ghbo5,Ghbo6,Ghbo7,Ghbo8,Ghbo9,Ghbo10,Ghbo11,Ghbo12,Ghbo13,Ghbo14,Ghbo15,Ghbo16,Ghbt1,Ghbt2,Ghbt3,Ghbt4,Ghbt5,Ghbt6,Ghbt7,Ghbt8,Ghbt9,Ghbt10,Ghbt11,Ghbt12,Ghbt13,Ghbt14,Ghbt15,Ghbt16,Koxy1,Koxy2,Koxy3,Koxy4,Koxy5,Koxy6,Koxy7,Koxy8,Koxy9,Koxy10,Koxy11,Koxy12,Koxy13,Koxy14,Koxy15,Koxy16,Khbr1,Khbr2,Khbr3,Khbr4,Khbr5,Khbr6,Khbr7,Khbr8,Khbr9,Khbr10,Khbr11,Khbr12,Khbr13,Khbr14,Khbr15,Khbr16,Khbo1,Khbo2,Khbo3,Khbo4,Khbo5,Khbo6,Khbo7,Khbo8,Khbo9,Khbo10,Khbo11,Khbo12,Khbo13,Khbo14,Khbo15,Khbo16,Khbt1,Khbt2,Khbt3,Khbt4,Khbt5,Khbt6,Khbt7,Khbt8,Khbt9,Khbt10,Khbt11,Khbt12,Khbt13,Khbt14,Khbt15,Khbt16
count,792.000000,792.000000,792.000000,792.000000,792.000000,792.000000,792.000000,756.000000,745.000000,739.000000,767.000000,792.000000,792.000000,791.000000,761.000000,789.000000,749.000000,792.000000,792.000000,788.000000,737.000000,560.000000,532.000000,754.000000,745.000000,739.000000,767.000000,792.000000,792.000000,791.000000,761.000000,789.000000,749.000000,792.000000,792.000000,788.000000,737.000000,560.000000,532.000000,754.000000,745.000000,739.000000,767.000000,792.000000,792.000000,791.000000,761.000000,789.000000,749.000000,792.000000,792.000000,788.000000,737.000000,560.000000,532.000000,754.000000,745.000000,739.000000,767.000000,792.000000,792.000000,791.000000,761.000000,789.000000,749.000000,792.000000,792.000000,788.000000,737.000000,560.000000,532.000000,752.000000,745.000000,738.000000,768.000000,792.000000,792.000000,792.000000,763.000000,788.000000,749.000000,790.000000,791.000000,787.000000,735.000000,558.000000,529.000000,752.000000,745.000000,738.000000,768.000000,792.000000,792.000000,792.000000,763.000000,788.000000,749.000000,790.000000,791.000000,787.000000,735.000000,558.000000,529.000000,752.000000,745.000000,738.000000,768.000000,792.000000,792.000000,792.000000,763.000000,788.000000,749.000000,790.000000,791.000000,787.000000,735.000000,558.000000,529.000000,752.000000,745.000000,738.000000,768.000000,792.000000,792.000000,792.000000,763.000000,788.000000,749.000000,790.000000,791.000000,787.000000,735.000000,558.000000,529.000000
mean,16.359848,31.431818,0.446970,1.501263,1098.820707,16.359848,29.897727,-0.036623,-0.033929,-0.046389,-0.021477,-0.002606,-0.005259,0.008652,0.018149,0.016723,0.023930,-0.000387,-0.014947,-0.024860,-0.022588,-0.004242,-0.006149,0.012769,0.029830,0.019443,0.014527,0.011471,0.011927,0.008588,-0.005312,0.003949,-0.003179,0.006680,0.021800,0.007450,0.025795,0.012187,0.020382,-0.023409,-0.004098,-0.026946,-0.006950,0.008865,0.006668,0.017241,0.012837,0.020672,0.020751,0.006293,0.006853,-0.017410,0.003208,0.007945,0.014233,-0.010639,0.025732,-0.007503,0.007578,0.020336,0.018595,0.025829,0.007524,0.024621,0.017572,0.012972,0.028654,-0.009959,0.029003,0.020133,0.034616,-0.017620,-0.027462,-0.040698,-0.023795,-0.019622,-0.005973,0.006983,0.011489,0.008561,0.016925,-0.002113,-0.005485,-0.020630,-0.029339,-0.003447,-0.010032,0.009758,0.021625,0.022688,0.011607,0.025648,0.010249,0.014409,0.006826,0.016902,0.016079,0.016201,0.015669,0.009543,0.024834,0.013943,0.016649,-0.007862,-0.005837,-0.018010,-0.012188,0.006026,0.004276,0.021391,0.018315,0.025464,0.033003,0.014088,0.010184,-0.011087,-0.004505,0.010496,0.006617,0.001896,0.015788,0.004678,-0.000581,0.031674,0.014525,0.035800,0.025141,0.042366,0.049082,0.030289,0.025853,-0.001545,0.020329,0.024439,0.023266
std,8.558746,17.258001,0.497494,0.500314,422.074287,8.558746,17.171210,0.318153,0.307532,0.354347,0.332360,0.298469,0.345603,0.375946,0.282086,0.333723,0.249005,0.425906,0.317527,0.525947,0.305340,0.453084,0.282680,0.167553,0.277243,0.149034,0.264758,0.153846,0.292892,0.249186,0.337103,0.168797,0.306298,0.217128,0.312820,0.260570,0.288509,0.320154,0.395311,0.246105,0.

In [ ]:
print(df_last_notLike.isna().sum().sum(),
df_last_Like.isna().sum().sum())

5177 6378


In [ ]:
Q1 = df_last[df_last.columns[9:]].quantile(0.25)
Q3 = df_last[df_last.columns[9:]].quantile(0.75)
IQR = Q3 - Q1
# print(Q1, Q3, IQR_notLike)

Q1_notLike = df_last_notLike[df_last_notLike.columns[9:]].quantile(0.25)
Q3_notLike = df_last_notLike[df_last_notLike.columns[9:]].quantile(0.75)
IQR_notLike = Q3_notLike - Q1_notLike
# print(Q1, Q3, IQR_notLike)

Q1_Like = df_last_Like[df_last_Like.columns[9:]].quantile(0.25)
Q3_Like = df_last_Like[df_last_Like.columns[9:]].quantile(0.75)
IQR_Like = Q3_Like - Q1_Like
# print(Q1, Q3, IQR_Like)

In [ ]:
# Outlier dataframe, True means it is an outlier

# total df
df_out = (df_last[df_last.columns[9:]] < (Q1 - 1.5 * IQR)) | (df_last[df_last.columns[9:]] > (Q3 + 1.5 * IQR))

true_count = ( df_out == True ).sum(axis=1).sum()
false_count = ( df_out == False ).sum(axis=1).sum()

print("True count: {} False Count: {}".format(true_count,false_count))

# split into like/not like

df_out_notLike = (df_last_notLike[df_last_notLike.columns[9:]] < (Q1_notLike - 1.5 * IQR_notLike)) | (df_last_notLike[df_last_notLike.columns[9:]] > (Q3_notLike + 1.5 * IQR_notLike))

true_count = ( df_out_notLike == True ).sum(axis=1).sum()
false_count = ( df_out_notLike == False ).sum(axis=1).sum()

print("True count: {} False Count: {}".format(true_count,false_count))

df_out_Like = (df_last_Like[df_last_Like.columns[9:]] < (Q1_Like - 1.5 * IQR_Like)) | (df_last_Like[df_last_Like.columns[9:]] > (Q3_Like + 1.5 * IQR_Like))

true_count = ( df_out_Like == True ).sum(axis=1).sum()
false_count = ( df_out_Like == False ).sum(axis=1).sum()

print("True count: {} False Count: {}".format(true_count,false_count))

True count: 10695 False Count: 182201
True count: 5210 False Count: 86310
True count: 5466 False Count: 95910


In [ ]:
# to check if all values of a row has outlier
print(df_out_notLike.shape, df_out_notLike[~df_out_notLike.all(1)].shape)
print(df_out_Like.shape, df_out_Like[~df_out_Like.all(1)].shape)

(715, 128) (715, 128)
(792, 128) (792, 128)


There are not any row with all values consist an outlier

In [ ]:
# Remove any row which has single or more outliers
df_last_mea = df_last[df_last.columns[9:]]
df_last_mea = df_last_mea.fillna(df_last_mea.mean())
df_last_out = df_last_mea[~((df_last_mea < (Q1 - 1.5 * IQR)) |(df_last_mea > (Q3 + 1.5 * IQR))).any(axis=1)]
print(df_last_mea.shape, df_last_out.shape)

(1507, 128) (641, 128)


### NOTLIKE Outlier Removal & Iterative Imp

In [ ]:
def outlierCountbyRow(df):
    
    Q1 = df[df.columns[9:]].quantile(0.25)
    Q3 = df[df.columns[9:]].quantile(0.75)
    IQR = Q3 - Q1
    
    df_last_mea = df[df.columns[9:]]
    df_last_mea = df_last_mea.fillna(df_last_mea.mean())
    outlier_matrix = ((df_last_mea < (Q1_notLike - 1.5 * IQR_notLike)) | (df_last_mea > (Q3_notLike + 1.5 * IQR_notLike)))
    
    outlier_count = outlier_matrix.sum(axis=0)
    outlier_percent = 100 * outlier_matrix.sum(axis=0) / df_last_mea.shape[0]
    
    # Df
    outlier_df = pd.concat([outlier_count, outlier_percent],
                              axis=1)
    
    outlier_df_final = outlier_df.rename(
        columns={0: 'Outlier Count', 1: '% of Total Values'})
    
    # Sort the table by percentage of outlier count descending
    outlier_df_final = (outlier_df_final[
        outlier_df_final.iloc[:, 1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1))
    return outlier_df_final

In [ ]:
outlierCountbyRow(df_last_notLike)

,Outlier Count,% of Total Values
Khbr10,84,11.7
Khbr12,75,10.5
Khbt1,69,9.7
Khbt7,68,9.5
Khbt11,68,9.5
...,...,...
Khbo5,20,2.8
Koxy5,19,2.7
Koxy15,18,2.5
Koxy7,16,2.2


In [ ]:
df_last_mea = df_last_notLike[df_last_notLike.columns[9:]]
df_last_mea = df_last_mea.fillna(df_last_mea.mean())
outlier_matrix_notLike = ((df_last_mea < (Q1_notLike - 1.5 * IQR_notLike)) | (df_last_mea > (Q3_notLike + 1.5 * IQR_notLike)))
# > 18 outlier count for each row corresponds to %15 of the notLike data to be removed
notlike_out_count = (outlier_matrix_notLike.sum(axis=1) >= 18).sum()
print(f'Row count to be removed:{notlike_out_count}, Total Rows: {df_last_mea.shape[0]} Removing Percentage:{(notlike_out_count/df_last_mea.shape[0]*100).round(1)}%')
df_last_mea = df_last_notLike[df_last_notLike.columns[9:]]

# Removing rows which have > 18 outliers
df_out_notLike = df_last_mea[outlier_matrix_notLike.sum(axis=1) < 18]

# Transforming the matrix as same as df_out_notlike to convert outliers into Nan 
outlier_matrix_notLike = outlier_matrix_notLike[outlier_matrix_notLike.sum(axis=1) < 18]
# df_out_notLike = func.outlier_removal(df_out_notLike)
df_out_notLike = df_out_notLike.mask((outlier_matrix_notLike))

Row count to be removed:102, Total Rows: 715 Removing Percentage:14.3%


In [ ]:
print(f'NaN percentage:%{df_out_notLike.isna().sum().sum()/(df_out_notLike[df_out_notLike.columns[9:]].shape[0]*df_out_notLike[df_out_notLike.columns[9:]].shape[1])*100}')

NaN percentage:%8.780347375491795


In [ ]:
df_out_notLike = pd.concat([df_last_notLike[df_last_notLike.columns[:9]], df_out_notLike], join='inner', axis=1)

In [ ]:
df_out_notLike

,Part_No,Stim_ID,Y/K,Direction,RespTime,Response,Like_OR_Nolike,G1,G2,Goxy1,Goxy2,Goxy3,Goxy4,Goxy5,Goxy6,Goxy7,Goxy8,Goxy9,Goxy10,Goxy11,Goxy12,Goxy13,Goxy14,Goxy15,Goxy16,Ghbr1,Ghbr2,Ghbr3,Ghbr4,Ghbr5,Ghbr6,Ghbr7,Ghbr8,Ghbr9,Ghbr10,Ghbr11,Ghbr12,Ghbr13,Ghbr14,Ghbr15,Ghbr16,Ghbo1,Ghbo2,Ghbo3,Ghbo4,Ghbo5,Ghbo6,Ghbo7,Ghbo8,Ghbo9,Ghbo10,Ghbo11,Ghbo12,Ghbo13,Ghbo14,Ghbo15,Ghbo16,Ghbt1,Ghbt2,Ghbt3,Ghbt4,Ghbt5,Ghbt6,Ghbt7,Ghbt8,Ghbt9,Ghbt10,Ghbt11,Ghbt12,Ghbt13,Ghbt14,Ghbt15,Ghbt16,Koxy1,Koxy2,Koxy3,Koxy4,Koxy5,Koxy6,Koxy7,Koxy8,Koxy9,Koxy10,Koxy11,Koxy12,Koxy13,Koxy14,Koxy15,Koxy16,Khbr1,Khbr2,Khbr3,Khbr4,Khbr5,Khbr6,Khbr7,Khbr8,Khbr9,Khbr10,Khbr11,Khbr12,Khbr13,Khbr14,Khbr15,Khbr16,Khbo1,Khbo2,Khbo3,Khbo4,Khbo5,Khbo6,Khbo7,Khbo8,Khbo9,Khbo10,Khbo11,Khbo12,Khbo13,Khbo14,Khbo15,Khbo16,Khbt1,Khbt2,Khbt3,Khbt4,Khbt5,Khbt6,Khbt7,Khbt8,Khbt9,Khbt10,Khbt11,Khbt12,Khbt13,Khbt14,Khbt15,Khbt16
1,1,16.0,0.0,2.0,1459.0,Right,False,1,2,-0.016150,0.089965,-0.148061,0.206889,0.070859,0.378170,0.412107,0.325312,0.439381,0.417096,0.303118,0.397769,0.629696,0.427812,0.483460,0.208160,-0.048277,-0.208151,0.119465,-0.232437,-0.011533,-0.272199,-0.206342,NaN,-0.276975,NaN,-0.143816,-0.315825,NaN,NaN,NaN,-0.316551,-0.064427,-0.118186,-0.028596,-0.025547,0.059326,0.105971,0.205765,-0.150861,0.162406,-0.158807,0.159301,0.081944,0.105952,-0.171707,0.054975,-0.108391,-0.112703,-0.326338,0.090868,-0.257984,0.047793,-0.166229,-0.000578,-0.627035,-0.114569,-0.734710,0.015485,-0.233881,-0.417792,NaN,-0.373509,-0.424942,0.174428,0.653096,-0.038715,0.650251,0.175557,NaN,0.819796,0.545715,0.706113,0.635638,0.500461,0.649931,0.412645,0.467408,0.471701,NaN,-0.249283,NaN,0.160259,NaN,-0.040307,NaN,-0.354216,NaN,-0.334903,NaN,-0.184560,-0.441936,-0.368731,NaN,-0.498745,NaN,-0.074855,-0.078230,0.121545,0.121389,0.135250,0.286725,0.465580,-0.160975,0.371210,-0.177202,0.315901,0.207995,0.043914,-0.237797,-0.027044,NaN,-0.324137,-0.809556,0.281804,-0.407472,0.094943,-0.193238,0.111364,-0.867665,0.036307,-0.990043,0.131340,-0.233941,-0.324817,-0.943003,-0.525789,NaN
5,1,13.0,1.0,2.0,1032.0,Right,False,1,6,-0.157769,-0.131881,-0.130702,-0.119899,0.183395,0.133742,0.663423,0.371626,0.649885,0.407279,0.481765,0.141346,0.053663,-0.086834,-0.090411,NaN,0.154087,-0.116972,0.097887,0.127719,0.004732,-0.079551,-0.238360,-0.067118,-0.121218,-0.010910,-0.137637,-0.069001,-0.002125,0.114249,0.125962,NaN,-0.003681,-0.248854,-0.032814,0.007820,0.188127,0.054190,0.425062,0.304508,0.528668,0.396369,0.344127,0.072345,0.051538,0.027414,0.035551,0.372527,0.150406,-0.365826,0.065073,0.135538,0.192858,-0.025361,0.186702,0.237391,0.407450,0.385459,0.206490,0.003344,0.049414,0.141663,0.161513,NaN,-0.170844,-0.019169,-0.502294,-0.204692,0.077855,0.150398,0.530801,0.542533,0.677680,0.492014,0.247811,0.161676,0.159329,0.297853,0.227202,-0.016945,0.246550,0.082843,0.397089,0.303013,0.198308,0.015846,-0.002846,-0.035761,-0.010398,0.071404,0.133901,-0.004075,0.078853,0.017547,0.064540,0.512409,0.075706,0.063673,-0.105206,0.098322,0.276163,0.166243,0.527955,0.506771,0.667283,0.563418,0.381712,0.157601,0.238182,0.315400,0.291742,0.495464,0.322256,0.146516,0.291883,0.401335,0.474471,0.182089,0.525109,0.471010,0.656885,0.634822,0.515613,0.153526,0.317035,0.332946,0.356282,1.007873
6,1,34.0,0.0,1.0,2176.0,Left,False,1,7,0.129763,0.109145,0.140136,0.202626,0.158629,0.226622,0.489568,0.261561,0.467585,0.325109,0.248852,0.281299,0.385314,0.260983,0.439901,NaN,0.020403,0.079393,-0.011015,-0.033796,0.068611,0.009992,-0.023461,0.111283,0.015690,0.086144,0.012579,0.017652,-0.094509,-0.021606,-0.151732,NaN,0.150166,0.188538,0.129121,0.168830,0.227241,0.236615,0.466106,0.372844,0.483275,0.411253,0.261431,0.298951,0.290804,0.239378,0.288169,-0.033526,0.170569,0.267931,0.118106,0.135034,0.295852,0.246607,0.442645,0.484127,0.498965,0.497397,0.274010,0.316602,0.196295,0.217772,0.136437,-0.536710,0.210062,0.082403,0.086784,0.129724,0.259321,0.324422,0.793394,0.447128,0.853688,0.542755,0.275723,0.494743,-0.277355,0.05

In [ ]:
def iter_imputation(df_cleaned):
    df_cleaned.reset_index(drop=True,inplace=True)
    data_iter =  df_cleaned.iloc[:,9:]
    # print total missing
    print('Missing: %d' % df_cleaned.isnull().sum().sum())
    # define imputer
    imputer = IterativeImputer(n_nearest_features=3, imputation_order='ascending')

    # fit and transform the dataset
    Xtrans = imputer.fit_transform(data_iter)
    # print total missing
    print('Missing: %d' % np.count_nonzero(np.isnan(Xtrans)))
    
    # Merging non-measurement features with filled measurement into df

    df_Xtrans= pd.DataFrame(Xtrans)             
    df_iter_imputation = pd.concat([df_cleaned[df_cleaned.columns[:9]], df_Xtrans], ignore_index=True, axis=1)
    df_iter_imputation.columns = df_cleaned.columns
    
    
    return df_iter_imputation

In [ ]:
# Some participants' hemodynamic measurements, 
# e.g. Goxy15, is totally empty so we need to first impute first row of it with its neighbours
# to not get eror in iterative imputation
part_no = df_out_notLike['Part_No'].drop_duplicates().to_list()
df_list = []
count = 1
for part in part_no:
    print(f'Iteration: {count} Part_no:{part}')
    df_temp = df_out_notLike[df_out_notLike['Part_No'] == part]
    # assign a value to the first value of a column where it has all values as NaN
    columns = df_temp.columns
    for colName in columns:
        if df_temp[colName].isna().sum() >= df_temp.shape[0]:
            
            if colName[-2:] == '15' or colName[-2:] == '16':
                df_temp.iloc[0,columns.get_loc(colName)] = df_temp.iloc[0,columns.get_loc(colName)-2:columns.get_loc(colName)].mean()
                if math.isnan(df_temp.iloc[0,columns.get_loc(colName)]):
                    df_temp.iloc[0,columns.get_loc(colName)] = df_temp.iloc[:,columns.get_loc(colName)-2:
                                                                            columns.get_loc(colName)].mean().mean()
            elif (int(colName[-1]) > 0 and int(colName[-1]) < 10) or (int(colName[-2:]) > 9 and int(colName[-2:]) < 14):
                df_temp.iloc[0,columns.get_loc(colName)] = df_temp.iloc[0,columns.get_loc(colName)+1:
                                                                    columns.get_loc(colName)+3].mean()
                if math.isnan(df_temp.iloc[0,columns.get_loc(colName)]):
                    df_temp.iloc[0,columns.get_loc(colName)] = df_temp.iloc[:,columns.get_loc(colName)+1:
                                                                    columns.get_loc(colName)+5].mean().mean()
            print(f'ColName={colName} and ParticipantNo={part}')

                    
                
    df_temp = func.iter_imputation(df_temp.reset_index(drop=True))
    df_list.append(df_temp)
    count += 1

Iteration: 1 Part_no:1
Missing: 63


/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


Missing: 0
Iteration: 2 Part_no:3
Missing: 98


/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


Missing: 0
Iteration: 3 Part_no:4
Missing: 320


...

/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Missing: 0
Iteration: 26 Part_no:28
ColName=Goxy15 and ParticipantNo=28
ColName=Ghbr15 and ParticipantNo=28
ColName=Ghbo15 and ParticipantNo=28
ColName=Ghbt15 and ParticipantNo=28
ColName=Koxy15 and ParticipantNo=28
ColName=Khbr15 and ParticipantNo=28
ColName=Khbo15 and ParticipantNo=28
ColName=Khbt15 and ParticipantNo=28
Missing: 130


/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Missing: 0
Iteration: 27 Part_no:29
ColName=Goxy3 and ParticipantNo=29
ColName=Ghbr3 and ParticipantNo=29
ColName=Ghbo3 and ParticipantNo=29
ColName=Ghbt3 and ParticipantNo=29
ColName=Koxy3 and ParticipantNo=29
ColName=Khbr3 and ParticipantNo=29
ColName=Khbo3 and ParticipantNo=29
ColName=Khbt3 and ParticipantNo=29
Missing: 251
Missing: 0


/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


In [ ]:
df_temp_2 = df_list.pop(0)
for part in df_list:
    df_temp_2 = pd.concat([df_temp_2,part])

In [ ]:
func.missing_values_table(df_temp_2)

Your selected dataframe has 137 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


### LIKE Outlier Removal & Iterative Imp

In [ ]:
outlierCountbyRow(df_last_Like)

,Outlier Count,% of Total Values
Ghbt11,90,11.4
Khbr6,84,10.6
Khbr10,80,10.1
Khbr8,77,9.7
Khbr12,77,9.7
...,...,...
Khbo6,21,2.7
Koxy3,21,2.7
Koxy1,20,2.5
Koxy9,18,2.3


In [ ]:
df_last_mea = df_last_Like[df_last_Like.columns[9:]]
df_last_mea = df_last_mea.fillna(df_last_mea.mean())
outlier_matrix_Like = ((df_last_mea < (Q1_Like - 1.5 * IQR_Like)) | (df_last_mea > (Q3_Like + 1.5 * IQR_Like)))
# > 18 outlier count for each row corresponds to %15 of the notLike data to be removed
like_out_count = (outlier_matrix_Like.sum(axis=1) >= 18).sum()
print(f'Row count to be removed:{like_out_count}, Total Rows: {df_last_mea.shape[0]} Removing Percentage:{(like_out_count/df_last_mea.shape[0]*100).round(1)}%')
df_last_mea = df_last_Like[df_last_Like.columns[9:]]

# Removing rows which have > 18 outliers
df_out_Like = df_last_mea[outlier_matrix_Like.sum(axis=1) < 18]

# Transforming the matrix as same as df_out_notlike to convert outliers into Nan 
outlier_matrix_Like = outlier_matrix_Like[outlier_matrix_Like.sum(axis=1) < 18]
# df_out_Like = func.outlier_removal(df_out_Like)
df_out_Like = df_out_Like.mask((outlier_matrix_Like))

Row count to be removed:105, Total Rows: 792 Removing Percentage:13.3%


In [ ]:
print(f'NaN percentage:%{df_out_Like.isna().sum().sum()/(df_out_Like[df_out_Like.columns[9:]].shape[0]*df_out_Like[df_out_Like.columns[9:]].shape[1])*100}')

NaN percentage:%9.184984037283035


In [ ]:
df_out_Like = pd.concat([df_last_Like[df_last_Like.columns[:9]], df_out_Like], join='inner', axis=1)

In [ ]:
# Some participants' hemodynamic measurements, 
# e.g. Goxy15, is totally empty so we need to first impute first row of it with its neighbours
# to not get eror in iterative imputation
part_no = df_out_Like['Part_No'].drop_duplicates().to_list()
df_list_2 = []
count = 1
for part in part_no:
    print(f'Iteration: {count} Part_no:{part}')
    df_temp = df_out_Like[df_out_Like['Part_No'] == part]
    # assign a value to the first value of a column where it has all values as NaN
    columns = df_temp.columns
    for colName in columns:
        if df_temp[colName].isna().sum() >= df_temp.shape[0]:
            
            if colName[-2:] == '15' or colName[-2:] == '16':
                df_temp.iloc[0,columns.get_loc(colName)] = df_temp.iloc[0,columns.get_loc(colName)-2:columns.get_loc(colName)].mean()
                if math.isnan(df_temp.iloc[0,columns.get_loc(colName)]):
                    df_temp.iloc[0,columns.get_loc(colName)] = df_temp.iloc[:,columns.get_loc(colName)-2:
                                                                            columns.get_loc(colName)].mean().mean()
            elif (int(colName[-1]) > 0 and int(colName[-1]) < 10) or (int(colName[-2:]) > 9 and int(colName[-2:]) < 14):
                df_temp.iloc[0,columns.get_loc(colName)] = df_temp.iloc[0,columns.get_loc(colName)+1:
                                                                    columns.get_loc(colName)+3].mean()
                if math.isnan(df_temp.iloc[0,columns.get_loc(colName)]):
                    df_temp.iloc[0,columns.get_loc(colName)] = df_temp.iloc[:,columns.get_loc(colName)+1:
                                                                    columns.get_loc(colName)+5].mean().mean()
            print(f'ColName={colName} and ParticipantNo={part}')

                    
                
    df_temp = func.iter_imputation(df_temp.reset_index(drop=True))
    df_list_2.append(df_temp)
    count += 1

Iteration: 1 Part_no:1
Missing: 44


/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


Missing: 0
Iteration: 2 Part_no:3
Missing: 146


/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


Missing: 0
Iteration: 3 Part_no:4
Missing: 225


...

/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Missing: 0
Iteration: 26 Part_no:28
ColName=Goxy15 and ParticipantNo=28
ColName=Ghbr15 and ParticipantNo=28
ColName=Ghbo15 and ParticipantNo=28
ColName=Ghbt15 and ParticipantNo=28
ColName=Koxy15 and ParticipantNo=28
ColName=Khbr15 and ParticipantNo=28
ColName=Khbo15 and ParticipantNo=28
ColName=Khbt15 and ParticipantNo=28
Missing: 474


/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Missing: 0
Iteration: 27 Part_no:29
ColName=Goxy3 and ParticipantNo=29
ColName=Ghbr3 and ParticipantNo=29
ColName=Ghbo3 and ParticipantNo=29
ColName=Ghbt3 and ParticipantNo=29
ColName=Koxy3 and ParticipantNo=29
ColName=Khbr3 and ParticipantNo=29
ColName=Khbo3 and ParticipantNo=29
ColName=Khbt3 and ParticipantNo=29
Missing: 209
Missing: 0


/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


In [ ]:
df_temp_3 = df_list_2.pop(0)
for part in df_list_2:
    df_temp_3 = pd.concat([df_temp_3,part])

In [ ]:
func.missing_values_table(df_temp_3)

Your selected dataframe has 137 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


### Merge Like and NotLike dfs

In [ ]:
df_out = pd.concat([df_temp_3, df_temp_2])

In [ ]:
df_out.sort_index(inplace=True)
df_out

,Part_No,Stim_ID,Y/K,Direction,RespTime,Response,Like_OR_Nolike,G1,G2,Goxy1,Goxy2,Goxy3,Goxy4,Goxy5,Goxy6,Goxy7,Goxy8,Goxy9,Goxy10,Goxy11,Goxy12,Goxy13,Goxy14,Goxy15,Goxy16,Ghbr1,Ghbr2,Ghbr3,Ghbr4,Ghbr5,Ghbr6,Ghbr7,Ghbr8,Ghbr9,Ghbr10,Ghbr11,Ghbr12,Ghbr13,Ghbr14,Ghbr15,Ghbr16,Ghbo1,Ghbo2,Ghbo3,Ghbo4,Ghbo5,Ghbo6,Ghbo7,Ghbo8,Ghbo9,Ghbo10,Ghbo11,Ghbo12,Ghbo13,Ghbo14,Ghbo15,Ghbo16,Ghbt1,Ghbt2,Ghbt3,Ghbt4,Ghbt5,Ghbt6,Ghbt7,Ghbt8,Ghbt9,Ghbt10,Ghbt11,Ghbt12,Ghbt13,Ghbt14,Ghbt15,Ghbt16,Koxy1,Koxy2,Koxy3,Koxy4,Koxy5,Koxy6,Koxy7,Koxy8,Koxy9,Koxy10,Koxy11,Koxy12,Koxy13,Koxy14,Koxy15,Koxy16,Khbr1,Khbr2,Khbr3,Khbr4,Khbr5,Khbr6,Khbr7,Khbr8,Khbr9,Khbr10,Khbr11,Khbr12,Khbr13,Khbr14,Khbr15,Khbr16,Khbo1,Khbo2,Khbo3,Khbo4,Khbo5,Khbo6,Khbo7,Khbo8,Khbo9,Khbo10,Khbo11,Khbo12,Khbo13,Khbo14,Khbo15,Khbo16,Khbt1,Khbt2,Khbt3,Khbt4,Khbt5,Khbt6,Khbt7,Khbt8,Khbt9,Khbt10,Khbt11,Khbt12,Khbt13,Khbt14,Khbt15,Khbt16
0,1,42.0,0.0,2.0,970.0,Left,True,1,3,-0.136447,-0.272634,-0.322293,-0.284869,-0.168651,-0.220061,-0.561465,-0.030628,-0.085240,0.279583,0.185449,0.248090,0.449046,-0.010317,0.395534,0.301781,0.115040,0.100586,0.119293,0.232511,0.163833,0.103896,0.329422,0.097941,0.002345,-0.065501,-0.122785,-0.246533,-0.138044,-0.383889,-0.125328,-0.081074,-0.021407,0.388075,-0.203000,-0.052358,-0.004818,-0.116165,-0.232043,0.067314,-0.082895,0.214081,0.062664,0.001557,0.311002,0.220147,0.270206,0.220707,0.093633,0.160414,-0.083707,0.180153,0.159015,-0.012269,0.097379,0.165255,-0.080550,0.148580,-0.060121,-0.244976,0.172957,-0.163742,0.144877,0.139633,0.282411,0.064885,0.454139,0.259362,0.431352,0.287118,0.094820,0.439876,-0.284129,0.363272,-0.219946,0.308470,-0.543672,0.077329,0.059039,0.083751,-0.152114,0.542451,-0.182078,0.009384,0.039956,-0.004640,0.229157,-0.068781,0.531488,0.114320,0.479081,-0.079169,0.637054,-0.129024,0.096418,-0.135218,0.130297,0.607335,0.272061,0.268746,0.471308,0.282478,0.323977,0.371096,0.247360,0.477592,0.259134,0.229301,0.093382,-0.051695,0.155457,-0.051467,-0.021817,0.911626,0.089983,0.278130,0.511265,0.277837,0.553134,0.302315,0.778848,0.591912,0.207387,0.150131,0.329199,-0.180719,0.251875,-0.186686
0,16,16.0,0.0,2.0,1721.0,Left,True,16,1,-0.068928,-0.018457,-0.003686,0.149468,-0.519452,-0.428884,-0.754657,-0.099173,-0.681682,-0.133129,-0.672642,-0.362823,0.890449,0.311610,0.601030,0.123866,-0.018018,-0.125408,-0.001474,-0.074776,0.230247,-0.020502,0.335108,-0.148566,0.248754,-0.127884,-0.234784,0.018552,-0.194957,-0.212682,-0.203819,-0.117211,-0.086946,-0.143865,-0.005160,-0.168225,-0.289205,-0.449385,-0.419549,-0.247739,-0.253706,-0.261013,-0.433852,-0.344272,0.695493,0.098928,0.397210,0.006656,-0.104964,-0.269273,-0.006634,-0.485918,-0.058958,-0.469887,-0.084441,-0.396305,0.174270,-0.388897,0.118011,-0.325720,0.029415,-0.113753,-0.113753,-0.110555,0.687412,0.150907,0.645137,-0.099942,0.348823,0.603910,0.559305,0.385108,0.271734,0.297862,0.117462,0.273418,0.678253,0.286875,0.482564,0.104864,-0.301511,-0.389820,-0.184714,-0.206939,-0.136328,-0.514524,-0.253366,-0.428671,-0.073838,-0.410229,0.025453,-0.434381,-0.195222,-0.376812,-0.286017,-0.173144,0.385901,0.463205,0.460422,0.168178,0.212496,0.089386,0.305939,-0.043563,0.197896,-0.112367,0.142915,-0.160963,0.483031,-0.089938,0.196547,-0.068280,0.084391,0.073385,0.275708,-0.412484,0.076168,-0.425138,0.052574,-0.472234,0.124057,-0.522596,0.168367,-0.595343,0.287809,-0.466750,-0.089470,-0.241423
0,21,38.0,0.0,1.0,991.0,Left,False,21,3,0.365581,0.524274,0.562633,0.479671,0.347419,0.283941,0.384466,0.116933,0.442167,0.087778,0.608343,0.094528,0.026687,0.286940,0.046732,0.312706,-0.225392,-0.105899,-0.140840,-0.207718,0.105128,-0.008259,0.063230,0.071133,-0.077034,0.017621,-0.067958,-0.079525,-0.192731,-0.127073,-0.243294,0.163011,0.468321,0.418375,0.421793,0.271953,0.452548,0.275682,0.447695,0.188066,0.365133,0.105399,0.540384,0.015003,0.237196,0.159867,0.172227,0.329483,0.323001,0.312476,0.280953,0.064235,0.152088,0.267423,0.510925,0.259199,0.288098,0.123020,0.472426,-0.064522,0.101383,0.

In [ ]:
df_out = df_out.reset_index(drop=True)

In [ ]:
df_iter = df_out.copy()

In [ ]:
df_iter

,Part_No,Stim_ID,Y/K,Direction,RespTime,Response,Like_OR_Nolike,G1,G2,Goxy1,Goxy2,Goxy3,Goxy4,Goxy5,Goxy6,Goxy7,Goxy8,Goxy9,Goxy10,Goxy11,Goxy12,Goxy13,Goxy14,Goxy15,Goxy16,Ghbr1,Ghbr2,Ghbr3,Ghbr4,Ghbr5,Ghbr6,Ghbr7,Ghbr8,Ghbr9,Ghbr10,Ghbr11,Ghbr12,Ghbr13,Ghbr14,Ghbr15,Ghbr16,Ghbo1,Ghbo2,Ghbo3,Ghbo4,Ghbo5,Ghbo6,Ghbo7,Ghbo8,Ghbo9,Ghbo10,Ghbo11,Ghbo12,Ghbo13,Ghbo14,Ghbo15,Ghbo16,Ghbt1,Ghbt2,Ghbt3,Ghbt4,Ghbt5,Ghbt6,Ghbt7,Ghbt8,Ghbt9,Ghbt10,Ghbt11,Ghbt12,Ghbt13,Ghbt14,Ghbt15,Ghbt16,Koxy1,Koxy2,Koxy3,Koxy4,Koxy5,Koxy6,Koxy7,Koxy8,Koxy9,Koxy10,Koxy11,Koxy12,Koxy13,Koxy14,Koxy15,Koxy16,Khbr1,Khbr2,Khbr3,Khbr4,Khbr5,Khbr6,Khbr7,Khbr8,Khbr9,Khbr10,Khbr11,Khbr12,Khbr13,Khbr14,Khbr15,Khbr16,Khbo1,Khbo2,Khbo3,Khbo4,Khbo5,Khbo6,Khbo7,Khbo8,Khbo9,Khbo10,Khbo11,Khbo12,Khbo13,Khbo14,Khbo15,Khbo16,Khbt1,Khbt2,Khbt3,Khbt4,Khbt5,Khbt6,Khbt7,Khbt8,Khbt9,Khbt10,Khbt11,Khbt12,Khbt13,Khbt14,Khbt15,Khbt16
0,1,42.0,0.0,2.0,970.0,Left,True,1,3,-0.136447,-0.272634,-0.322293,-0.284869,-0.168651,-0.220061,-0.561465,-0.030628,-0.085240,0.279583,0.185449,0.248090,0.449046,-0.010317,0.395534,0.301781,0.115040,0.100586,0.119293,0.232511,0.163833,0.103896,0.329422,0.097941,0.002345,-0.065501,-0.122785,-0.246533,-0.138044,-0.383889,-0.125328,-0.081074,-0.021407,0.388075,-0.203000,-0.052358,-0.004818,-0.116165,-0.232043,0.067314,-0.082895,0.214081,0.062664,0.001557,0.311002,0.220147,0.270206,0.220707,0.093633,0.160414,-0.083707,0.180153,0.159015,-0.012269,0.097379,0.165255,-0.080550,0.148580,-0.060121,-0.244976,0.172957,-0.163742,0.144877,0.139633,0.282411,0.064885,0.454139,0.259362,0.431352,0.287118,0.094820,0.439876,-0.284129,0.363272,-0.219946,0.308470,-0.543672,0.077329,0.059039,0.083751,-0.152114,0.542451,-0.182078,0.009384,0.039956,-0.004640,0.229157,-0.068781,0.531488,0.114320,0.479081,-0.079169,0.637054,-0.129024,0.096418,-0.135218,0.130297,0.607335,0.272061,0.268746,0.471308,0.282478,0.323977,0.371096,0.247360,0.477592,0.259134,0.229301,0.093382,-0.051695,0.155457,-0.051467,-0.021817,0.911626,0.089983,0.278130,0.511265,0.277837,0.553134,0.302315,0.778848,0.591912,0.207387,0.150131,0.329199,-0.180719,0.251875,-0.186686
1,16,16.0,0.0,2.0,1721.0,Left,True,16,1,-0.068928,-0.018457,-0.003686,0.149468,-0.519452,-0.428884,-0.754657,-0.099173,-0.681682,-0.133129,-0.672642,-0.362823,0.890449,0.311610,0.601030,0.123866,-0.018018,-0.125408,-0.001474,-0.074776,0.230247,-0.020502,0.335108,-0.148566,0.248754,-0.127884,-0.234784,0.018552,-0.194957,-0.212682,-0.203819,-0.117211,-0.086946,-0.143865,-0.005160,-0.168225,-0.289205,-0.449385,-0.419549,-0.247739,-0.253706,-0.261013,-0.433852,-0.344272,0.695493,0.098928,0.397210,0.006656,-0.104964,-0.269273,-0.006634,-0.485918,-0.058958,-0.469887,-0.084441,-0.396305,0.174270,-0.388897,0.118011,-0.325720,0.029415,-0.113753,-0.113753,-0.110555,0.687412,0.150907,0.645137,-0.099942,0.348823,0.603910,0.559305,0.385108,0.271734,0.297862,0.117462,0.273418,0.678253,0.286875,0.482564,0.104864,-0.301511,-0.389820,-0.184714,-0.206939,-0.136328,-0.514524,-0.253366,-0.428671,-0.073838,-0.410229,0.025453,-0.434381,-0.195222,-0.376812,-0.286017,-0.173144,0.385901,0.463205,0.460422,0.168178,0.212496,0.089386,0.305939,-0.043563,0.197896,-0.112367,0.142915,-0.160963,0.483031,-0.089938,0.196547,-0.068280,0.084391,0.073385,0.275708,-0.412484,0.076168,-0.425138,0.052574,-0.472234,0.124057,-0.522596,0.168367,-0.595343,0.287809,-0.466750,-0.089470,-0.241423
2,21,38.0,0.0,1.0,991.0,Left,False,21,3,0.365581,0.524274,0.562633,0.479671,0.347419,0.283941,0.384466,0.116933,0.442167,0.087778,0.608343,0.094528,0.026687,0.286940,0.046732,0.312706,-0.225392,-0.105899,-0.140840,-0.207718,0.105128,-0.008259,0.063230,0.071133,-0.077034,0.017621,-0.067958,-0.079525,-0.192731,-0.127073,-0.243294,0.163011,0.468321,0.418375,0.421793,0.271953,0.452548,0.275682,0.447695,0.188066,0.365133,0.105399,0.540384,0.015003,0.237196,0.159867,0.172227,0.329483,0.323001,0.312476,0.280953,0.064235,0.152088,0.267423,0.510925,0.259199,0.288098,0.123020,0.472426,-0.064522,0.101383,0.

In [ ]:
#Relocate some columns
cols = df_iter.columns.tolist()
cols = cols[0:6] + cols [7:9] + cols[6:7] + cols[9:]
df_iter = df_iter.copy()[cols]

# Reducing Columns to 8

## Reducing

In [ ]:
df_AvgIter = func.avg_lr(df_iter)

In [ ]:
df_AvgIter.replace({True: 1, False : 0}, inplace=True)

In [ ]:
df_AvgIter

,Part_No,Stim_ID,Y/K,Direction,RespTime,Response,G1,G2,Like_OR_Nolike,Goxy_Avg,Ghbr_Avg,Ghbo_Avg,Ghbt_Avg,Koxy_Avg,Khbr_Avg,Khbo_Avg,Khbt_Avg
0,1,42.0,0.0,2.0,970.0,Left,1,3,1,-1.132161,-1.087621,-0.143251,13.411618,-14.843944,0.272837,2.006748,1.494619
1,16,16.0,0.0,2.0,1721.0,Left,16,1,1,-22.740972,-0.214326,19.142863,2.566926,1.305459,1.255527,3.467762,0.559877
2,21,38.0,0.0,1.0,991.0,Left,21,3,0,1.608138,0.739094,1.529820,1.006081,1.525001,0.599705,1.256539,1.486862
3,3,25.0,1.0,1.0,939.0,Left,3,3,0,1.656311,1.604235,1.345846,1.616410,-0.745450,1.918734,0.366984,0.824065
4,5,43.0,1.0,1.0,1280.0,Right,5,1,1,0.396326,0.223273,0.446869,0.394131,-0.479761,0.931197,0.281010,0.587093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,28,46.0,0.0,2.0,742.0,Left,28,60,1,39.610772,4.283991,-1.854211,1.460212,-7.937898,5.669803,-0.344697,1.809424
1296,26,50.0,0.0,2.0,651.0,Left,26,57,1,1.180518,0.538748,1.178777,1.274994,1.135423,0.986895,1.282455,1.173443
1297,26,57.0,1.0,2.0,850.0,Left,26,58,1,0.780830,0.997435,-0.205744,1.242661,0.326112,1.650316,-1.666438,2.286108
1298,26,53.0,1.0,1.0,1129.0,Right,26,59,1,0.066055,0.222293,-0.831823,0.482801,0.123929,0.372370,-1.058195,0.590255


## Outlier Removal

In [ ]:
df_AvgIter_notLike = df_AvgIter[df_AvgIter['Like_OR_Nolike'] == 0]
df_AvgIter_Like = df_AvgIter[df_AvgIter['Like_OR_Nolike'] == 1]

In [ ]:
Q1_notLike = df_AvgIter_notLike[df_AvgIter_notLike.columns[9:]].quantile(0.25)
Q3_notLike = df_AvgIter_notLike[df_AvgIter_notLike.columns[9:]].quantile(0.75)
IQR_notLike = Q3_notLike - Q1_notLike
# print(Q1, Q3, IQR_notLike)

Q1_Like = df_AvgIter_Like[df_AvgIter_Like.columns[9:]].quantile(0.25)
Q3_Like = df_AvgIter_Like[df_AvgIter_Like.columns[9:]].quantile(0.75)
IQR_Like = Q3_Like - Q1_Like
# print(Q1, Q3, IQR_Like)

In [ ]:
# split into like/not like
df_out_notLike = (df_AvgIter_notLike[df_AvgIter_notLike.columns[9:]] < (Q1_notLike - 1.5 * IQR_notLike)) | (df_AvgIter_notLike[df_AvgIter_notLike.columns[9:]] > (Q3_notLike + 1.5 * IQR_notLike))
true_count = ( df_out_notLike == True ).sum(axis=1).sum()
false_count = ( df_out_notLike == False ).sum(axis=1).sum()
print("True count: {} False Count: {}".format(true_count,false_count))

df_out_Like = (df_AvgIter_Like[df_AvgIter_Like.columns[9:]] < (Q1_Like - 1.5 * IQR_Like)) | (df_AvgIter_Like[df_AvgIter_Like.columns[9:]] > (Q3_Like + 1.5 * IQR_Like))
true_count = ( df_out_Like == True ).sum(axis=1).sum()
false_count = ( df_out_Like == False ).sum(axis=1).sum()
print("True count: {} False Count: {}".format(true_count,false_count))

True count: 799 False Count: 4105
True count: 801 False Count: 4695


In [ ]:
df_AvgIter_notLike.shape
df_AvgIter_Like.shape

(613, 17)

(687, 17)

In [ ]:
df_AvgIter_notLike_out = func.outlier_removal(df_AvgIter_notLike)
df_AvgIter_Like_out = func.outlier_removal(df_AvgIter_Like)

True count: 799 False Count: 4105
(613, 8) (613, 8)
True count: 801 False Count: 4695
(687, 8) (687, 8)


In [ ]:
df_AvgIter_Like_out[df_AvgIter_Like_out['Like_OR_Nolike'] == False].count()

Part_No           0
Stim_ID           0
Y/K               0
Direction         0
RespTime          0
Response          0
G1                0
G2                0
Like_OR_Nolike    0
Goxy_Avg          0
Ghbr_Avg          0
Ghbo_Avg          0
Ghbt_Avg          0
Koxy_Avg          0
Khbr_Avg          0
Khbo_Avg          0
Khbt_Avg          0
dtype: int64

## Iterative Imputing by 0 and 1 Separately

In [ ]:
#@title Varsayılan başlık metni
part_no = df_AvgIter_notLike_out['Part_No'].drop_duplicates().to_list()
df_list = []
count = 1
for part in part_no:
    print(f'Iteration: {count}')
    df_temp = df_AvgIter_notLike_out[df_AvgIter_notLike_out['Part_No'] == part]
    # assign a value to the first value of a column where it has all values as NaN
    columns = df_temp.columns
    for colName in columns:
        if df_temp[colName].isna().sum() >= df_temp.shape[0]:
            
            if colName[-2:] == '15' or colName[-2:] == '16':
                df_temp.iloc[0,columns.get_loc(colName)] = df_temp.iloc[0,columns.get_loc(colName)-2:columns.get_loc(colName)].mean()
                if math.isnan(df_temp.iloc[0,columns.get_loc(colName)]):
                    df_temp.iloc[0,columns.get_loc(colName)] = df_temp.iloc[:,columns.get_loc(colName)-2:
                                                                            columns.get_loc(colName)].mean().mean()
            elif (int(colName[-1]) > 0 and int(colName[-1]) < 10) or (int(colName[-2:]) > 9 and int(colName[-2:]) < 14):
                df_temp.iloc[0,columns.get_loc(colName)] = df_temp.iloc[0,columns.get_loc(colName)+1:
                                                                    columns.get_loc(colName)+3].mean()
                if math.isnan(df_temp.iloc[0,columns.get_loc(colName)]):
                    df_temp.iloc[0,columns.get_loc(colName)] = df_temp.iloc[:,columns.get_loc(colName)+1:
                                                                    columns.get_loc(colName)+5].mean().mean()
            print(f'ColName={colName} and ParticipantNo={part}')

                    
                
    df_temp = iter_imputation(df_temp.reset_index(drop=True))
    df_list.append(df_temp)
    count += 1

Iteration: 1
Missing: 29
Missing: 0
Iteration: 2
Missing: 22


/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


Missing: 0
Iteration: 3
Missing: 31
Missing: 0
Iteration: 4
Missing: 44


/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


Missing: 0
Iteration: 5
Missing: 23
Missing: 0
Iteration: 6
Missing: 35


...

/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


Missing: 0
Iteration: 25
Missing: 34
Missing: 0
Iteration: 26
Missing: 18


/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


Missing: 0
Iteration: 27
Missing: 36
Missing: 0


/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


In [ ]:
df_temp_AvgIter_notLike_out = df_list.pop(0)
for part in df_list:
    df_temp_AvgIter_notLike_out = pd.concat([df_temp_AvgIter_notLike_out,part])

In [ ]:
part_no = df_AvgIter_Like_out['Part_No'].drop_duplicates().to_list()
df_list_2 = []
count = 1
for part in part_no:
    print(f'Iteration: {count}')
    df_temp = df_AvgIter_Like_out[df_AvgIter_Like_out['Part_No'] == part]
    # assign a value to the first value of a column where it has all values as NaN
    columns = df_temp.columns
    for colName in columns:
        if df_temp[colName].isna().sum() >= df_temp.shape[0]:
            
            if colName[-2:] == '15' or colName[-2:] == '16':
                df_temp.iloc[0,columns.get_loc(colName)] = df_temp.iloc[0,columns.get_loc(colName)-2:columns.get_loc(colName)].mean()
                if math.isnan(df_temp.iloc[0,columns.get_loc(colName)]):
                    df_temp.iloc[0,columns.get_loc(colName)] = df_temp.iloc[:,columns.get_loc(colName)-2:
                                                                            columns.get_loc(colName)].mean().mean()
            elif (int(colName[-1]) > 0 and int(colName[-1]) < 10) or (int(colName[-2:]) > 9 and int(colName[-2:]) < 14):
                df_temp.iloc[0,columns.get_loc(colName)] = df_temp.iloc[0,columns.get_loc(colName)+1:
                                                                    columns.get_loc(colName)+3].mean()
                if math.isnan(df_temp.iloc[0,columns.get_loc(colName)]):
                    df_temp.iloc[0,columns.get_loc(colName)] = df_temp.iloc[:,columns.get_loc(colName)+1:
                                                                    columns.get_loc(colName)+5].mean().mean()
            print(f'ColName={colName} and ParticipantNo={part}')

                    
                
    df_temp = iter_imputation(df_temp.reset_index(drop=True))
    df_list_2.append(df_temp)
    count += 1

Iteration: 1
Missing: 23
Missing: 0
Iteration: 2
Missing: 44


/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


Missing: 0
Iteration: 3
Missing: 16
Missing: 0
Iteration: 4
Missing: 74


/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


Missing: 0
Iteration: 5
Missing: 21
Missing: 0
Iteration: 6
Missing: 29


...

/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


Missing: 0
Iteration: 25
Missing: 25
Missing: 0
Iteration: 26
Missing: 40


/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


Missing: 0
Iteration: 27
Missing: 17
Missing: 0


/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


In [ ]:
df_temp_AvgIter_Like_out = df_list_2.pop(0)
for part in df_list_2:
    df_temp_AvgIter_Like_out = pd.concat([df_temp_AvgIter_Like_out,part])

In [ ]:
# df_temp_AvgIter_Like_out[df_temp_AvgIter_Like_out['Like_OR_Nolike'] == True].count()
# df_temp_AvgIter_notLike_out[df_temp_AvgIter_notLike_out['Like_OR_Nolike'] == False].count()

In [ ]:
df_AvgIter = pd.concat([df_temp_AvgIter_Like_out, df_temp_AvgIter_notLike_out])

In [ ]:
df_AvgIter.sort_values(by=['Part_No'], inplace=True)

In [ ]:
df_AvgIter.reset_index(inplace=True, drop = True)

In [ ]:
df_AvgIter

,Part_No,Stim_ID,Y/K,Direction,RespTime,Response,G1,G2,Like_OR_Nolike,Goxy_Avg,Ghbr_Avg,Ghbo_Avg,Ghbt_Avg,Koxy_Avg,Khbr_Avg,Khbo_Avg,Khbt_Avg
0,1,42.0,0.0,2.0,970.0,Left,1,3,1,-0.960737,-0.859666,-0.143251,0.509734,0.530021,0.272837,2.006748,1.213394
1,1,4.0,0.0,2.0,1529.0,Right,1,56,0,0.218204,3.049154,0.668902,1.004436,0.369163,2.091083,0.742004,1.294716
2,1,21.0,1.0,2.0,1418.0,Right,1,55,0,0.675076,0.257604,0.933370,1.154278,1.084411,0.120118,1.292939,1.616540
3,1,10.0,0.0,2.0,1699.0,Right,1,53,0,0.116797,-0.033963,1.452581,-0.223084,0.121432,0.449205,1.872797,0.673632
4,1,5.0,1.0,1.0,1180.0,Left,1,50,0,2.456716,0.522279,1.976112,1.732519,1.806730,-0.122283,1.656707,1.528336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,29,9.0,1.0,2.0,1779.0,Right,29,45,0,3.177237,0.537894,2.054040,0.949882,0.453014,0.432891,0.478905,0.272331
1296,29,4.0,0.0,1.0,891.0,Left,29,44,0,-0.366805,1.716181,0.776050,0.614703,0.070771,1.248599,-0.352277,0.523548
1297,29,16.0,0.0,1.0,1021.0,Left,29,43,0,1.144270,0.535840,0.882389,0.607945,2.505372,-1.176549,0.968134,0.336445
1298,29,12.0,0.0,1.0,998.0,Left,29,39,0,0.412362,-0.151754,0.985203,0.654896,-0.288076,2.263285,0.078231,0.523179


# Export Data

In [ ]:
# Finally transform boolean expression True, False into 1 0 for imputation and save it
df_iter.replace({True: 1, False : 0}, inplace=True)
df_iter.to_csv("C:\\Users\\Lenovo\\Desktop\\ML Neuro\\data\\df_iter_01.csv", index=False)

In [ ]:
df_AvgIter.to_csv("C:\\Users\\Lenovo\\Desktop\\ML Neuro\\data\\df_AvgIter_01.csv", index=False)